# Prosept. Создание сервиса для полуавтоматической разметки товаров


In [1]:
import pandas as pd
import numpy as np

import sklearn
import scipy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix

import re

In [2]:
print(re.__version__)

2.2.1


In [3]:
pd.set_option('display.max_colwidth', None)
pd.options.mode.chained_assignment = None

## Введение

Цели данного этапа: разработка прототипа модели и скорейшая проработка важных вопросов, таких как общий вид интеграции в backend. После того, как данное решение пройдёт полный цикл от разбработки до проверки, команда DS приступит к решению дополнительных вопросов, вынесеных в список к проработке в дальнейшем и улучшению метрики. <br>
За основную гипотезу выбрано предположение о том, что названий товаров достаточно для получения мэтча с приемлемой точностью. Для реализации этой гипотезы выбран метод `cosine_similarity`, применённый к векторам названий, полученным с помощью `TfidfVectorizer()`. Сопоставляются вектора столбцов `df_dealerprice.product_name` и `df_product.name`. Остальные фичи не используются.  Необходимость их добавления в проект будет рассмотрена по итогу реализации данного решения и его оценки.<br>
Так же сразу стоит отметить, что по договорённости с бэкендом наш модуль вызвается один раз при запуске программы, просчитывает все мэтчи и выдаёт таблицу с мэтчами, с которой в дальнейшем и работает бэкенд. Функция просчёта мэтчей (самая ресурсоёмкая операция) в тетрадке выполняется за **1.64** секунды, что говорит о том, что и в сборке выполнение данного кода не должно приводить к задержкам и неудобствам для оператора.

## Подготовка данных

Загрузим данные их датасетов `marketing_dealerprice.csv`, `marketing_product.csv` и `marketing_productdealerkey.csv`. `marketing_dealer` не нужен для DS модуля, так как информация из него подтягивается бэкендом.

Датасеты `marketing_dealerprice.csv` и `marketing_product.csv` сразу очистим от дубликатов, чтобы не повторять расчёт для одинаковых позиций. Стоит отметить, что в датасетах обнаружены одинаковые названия с разными `id`, а также пропуски. Данные вопросы вынесены в список к проработке в дальнейшем. <br>
Так же для `df_product` создан список уникальных и упорядоченных `id_unique`. Данная операция осталась от предыдущего метода совмещения исходных данных, который за 19 часов до дэдлайна оказался нежизнеспособным. В дальнейшем будет рассмотрена возможность упразднения данной операции.

In [4]:
path_to_dir = '../data/'

In [5]:
df_dealerprice = pd.read_csv(path_to_dir + 'marketing_dealerprice.csv', sep=';')
df_dealerprice[['product_key', 'product_name']].nunique()

product_key     1965
product_name    1953
dtype: int64

In [6]:
df_dealerprice = df_dealerprice[['product_key', 'product_name']]
df_dealerprice.drop_duplicates(inplace=True)
df_dealerprice.head(2)

,product_key,product_name
0,546227,"Средство универсальное Prosept Universal Spray, 500мл"
1,546408,"Концентрат Prosept Multipower для мытья полов, цитрус 1л"


In [7]:
df_product = pd.read_csv(path_to_dir + 'marketing_product.csv', sep=';')
df_product[['id', 'name']].nunique()

id      496
name    487
dtype: int64

In [8]:
df_product = df_product[['id', 'name']]
df_product.dropna(inplace=True)
df_product.drop_duplicates(subset='name', inplace=True)

df_product['id_unique'] = pd.factorize(df_product['name'])[0]
df_product.set_index('id_unique', inplace=True)

df_product.head(2)

,id,name
id_unique,,
0,245,Антисептик невымываемыйPROSEPT ULTRAконцентрат 1:10 / 1 л
1,3,Антигололед - 32 PROSEPTготовый состав / 12 кг


In [9]:
df_productdealerkey = pd.read_csv(path_to_dir + 'marketing_productdealerkey.csv', sep=';')
df_productdealerkey.head()

,id,key,dealer_id,product_id
0,1,546227,2,12
1,2,651265,2,106
2,3,546257,2,200
3,4,546408,2,38
4,5,651258,2,403


Данные готовы, можно приступить к предобработке текста.

## Предобработка текста

**Данный раздел находится в разработке. Количество функций будет сокращено, код оптимизирован.**

Были выявлены следующие недостатки в данных:

1. Разный регистр букв
2. Расхождение единиц объёма/веса
3. Слитное написание слов, отсутствие пробелов. Некоторые записи с полным отсутствием пробелов определяются как одно слово, что даёт `cos_sim = 0` до всех предложеных позиций.

Основная идея - привести все записи и слова к одному формату, чтобы увеличить количество совпадений по словам и снизить размерность векторного пространства для более точного определения `cos_sim`. На данный момент удалось снизить размерность вектора с 1222 слов до 1099.

In [10]:
corpus = pd.concat([df_product.name, df_dealerprice.product_name]).drop_duplicates().values
corpus

array(['Антисептик невымываемыйPROSEPT ULTRAконцентрат 1:10  / 1 л',
       'Антигололед - 32 PROSEPTготовый состав / 12 кг',
       'Герметик акриловый цвет сосна, ф/п 600мл', ...,
       'Средство для удаления клейкой ленты, клея, наклеек 0,4л PROSEPT Duty Universal готовый состав',
       'Отбеливатель для древесины PROSEPT ECO 50 готовый состав 1 кг',
       'Герметик акриловый межшовный для деревянных конструкций, цвет "Орех" готовый состав 0,6 кг'],
      dtype=object)

In [11]:
# Разделим слова в тексте

def split_compound_text(text):
    pattern = re.compile(r'(?<=[а-яА-Я])(?=[A-Za-z])|(?<=[A-Za-z])(?=[а-яА-Я])')
    splitted_text = pattern.sub(" ", str(text))
    return splitted_text

In [12]:
df_dealerprice.product_name = df_dealerprice.product_name.apply(split_compound_text)
df_product.name = df_product.name.apply(split_compound_text)

In [13]:
# Заменим "мл" на "л" и разделеним на 1000, если в миллилитрах

def convert_milliliters(text):
    return re.sub(r'(\d+)\s*мл', lambda x: str(int(x.group(1)) / 1000) +' '+'л', text)

In [14]:
df_dealerprice.product_name = df_dealerprice.product_name.apply(lambda x: convert_milliliters(x))
df_product.name = df_product.name.apply(lambda x: convert_milliliters(x))

In [15]:
# Удалим все лишние знаки

def clear_text(text):
    cleaned_text = re.sub(r'[!#$%&\'()*+,./:;<=>?@[\]^_`{|}~—\"\\-]+', ' ', text)
    return cleaned_text

In [16]:
df_dealerprice.product_name = df_dealerprice.product_name.apply(clear_text)
df_product.name = df_product.name.apply(clear_text)

In [17]:
# Приведём всё к нижнему регистру

df_dealerprice.product_name = df_dealerprice.product_name.apply(lambda x: x.lower())
df_product.name = df_product.name.apply(lambda x: x.lower())

In [18]:
# Разделим
# Чистящеесредство , Средстводляпосудомоечной, Средстводлямытьяполов, Средстводляудаленияржавчины,
# Чистящеесредстводлябаниисауны, Средстводляочисткилюстр, Средстводляудаленияцемента

df_dealerprice.product_name = df_dealerprice.product_name.apply(lambda x: re.sub(r'(средство|мытья|для|чистящее|удаления|очистки)', r'\1 ', str(x)))
df_product.name = df_product.name.apply(lambda x: re.sub(r'(средство|мытья|для|чистящее|удаления|очистки)', r'\1 ', str(x)))

In [19]:
# Удалим и, для, д, с, ф, п, ая, 007, i

df_dealerprice.product_name = df_dealerprice.product_name.str.replace(r'\b(?:и|для|д|с|ф|п|ая|007|i)\b', '', regex=True)
df_product.name = df_product.name.str.replace(r'\b(?:и|для|д|с|ф|п|ая|007|i)\b', '', regex=True)

Объединим два корпуса названий продуктов в один.

In [20]:
corpus = pd.concat([df_product.name, df_dealerprice.product_name]).drop_duplicates().values
corpus

array(['антисептик невымываемый prosept ultra концентрат 1 10    1 л',
       'антигололед   32 prosept готовый состав   12 кг',
       'герметик акриловый цвет сосна    0 6 л', ...,
       'средство    удаления  клейкой ленты  клея  наклеек 0 4л prosept duty universal готовый состав',
       'отбеливатель   древесины prosept eco 50 готовый состав 1 кг',
       'герметик акриловый межшовный   деревянных конструкций  цвет  орех  готовый состав 0 6 кг'],
      dtype=object)

Предобработка проведена. Её успешность оценивалась по размерности получаемого через `TfidfVectorizer()` вектора, о чём было сказано ранее и по метрике, о чём будет сказано в дальнейшем, дабы не раздувать код дополнительными демонстрациями. Приступим к `TF-IDF` и `cos_sim` и созданию функции предсказания.

## `TF-IDF` и `cos_sim`

Для сопоставления названий от дилеров названиям от заказчика выбран метод `cosine_similarity`, применённый к векторам названий, полученным с помощью `TfidfVectorizer()`.

In [21]:
vectorizer = TfidfVectorizer()
vectorizer_fited = vectorizer.fit(corpus)

Параллельно посмотрим на размерность вектора.

In [22]:
vector_name = vectorizer.transform(df_product.name)
len(vector_name.todense().tolist()[0])

1099

В функции предсказания выполняется итерирование по `vectors_feat`, рассчитывается `cosine_similarity` для каждой пары `vectors_feat[i], vectors_targ`. Функция возвращает лучшие `n` позиций вместе с соответствующими `cosine_similarity`. Для оптимизации времени расчёта матрицы векторов конвертированы в формат `Compressed Sparse`.

In [23]:
%%time

def prediction(feat, targ, tf_idf, n):
    vectors_targ = tf_idf.transform(targ).tocsc()
    vectors_feat = tf_idf.transform(feat).tocsr()

    pred = np.zeros((len(feat), n), dtype=int)
    pred_sim = np.zeros((len(feat), n), dtype=float)

    for i in range(len(feat)):
        cos_sim = cosine_similarity(vectors_feat[i], vectors_targ)
        top_n_indexes = np.argsort(cos_sim)[0, -n:][::-1]
        top_n_values = cos_sim[0, top_n_indexes]

        pred[i] = top_n_indexes
        pred_sim[i] = list(np.around(np.array(top_n_values),6))

    return pred, pred_sim

pred, pred_sim = prediction(df_dealerprice.product_name, df_product.name, vectorizer_fited, 5)

CPU times: total: 1.78 s
Wall time: 1.78 s


**1.7** секунды - неплохой результат.

In [24]:
print(pred[0:5])

[[335 479 176 438 427]
 [ 63 174  89 263 262]
 [211 213 208 356 353]
 [458 247 276 144 192]
 [244 198 210 219 401]]


In [25]:
print(pred_sim[0:5])

[[0.71266  0.71266  0.635631 0.457258 0.457258]
 [0.760274 0.758144 0.675339 0.617515 0.617515]
 [0.928725 0.650948 0.650948 0.180814 0.159013]
 [0.700914 0.700914 0.66071  0.188307 0.165063]
 [0.910428 0.608733 0.466645 0.466645 0.397241]]


Функция работает и выдаёт корректный результат. Стоит отметить, что первая версия функции предсказания выполнялась за **~70** секунд и выдавала дублирующиеся значения айди названий от заказчика, то есть в топ-5 могло попасть всего 3 уникальных айди, что ухудшало не только метрику, но и выдачу для оператора в дальнейшем. <br>
На данном этапе мы получили список предложенных нашей функцией предсказания позиций заказчика для каждой позиции дилера. Однако данные индексы не соответствуют индексам во входных файлах. Необходимо сопоставить их настоящим `product_id` и `product_key`.

## Сопоставление нашего расчёта с `product_id` и `product_key`.

Напишем код для сопоставления айди.

In [26]:
df_dealerprice.head(3)

,product_key,product_name
0,546227,средство универсальное prosept universal spray 0 5 л
1,546408,концентрат prosept multipower мытья полов цитрус 1л
2,546234,средство чистки люстр prosept universal anti dust 0 5 л


In [27]:
df_product.head(3)

,id,name
id_unique,,
0,245,антисептик невымываемый prosept ultra концентрат 1 10 1 л
1,3,антигололед 32 prosept готовый состав 12 кг
2,443,герметик акриловый цвет сосна 0 6 л


In [28]:
%%time

result = []

for row in pred:
    new_row = []
    for item in row:
        try:
            new_row.append(df_product.loc[item, 'id'].values[0])
        except:
            new_row.append(df_product.loc[item, 'id'])
    result.append(new_row)

CPU times: total: 156 ms
Wall time: 152 ms


Создадим датафрейм, с истинными `product_id` и `product_key`.

In [29]:
df_result = df_dealerprice['product_key']
df_result = df_result.to_frame()
df_result['product_id'] = result
df_result['pred_sim'] = pred_sim.tolist()
df_result.head()

,product_key,product_id,pred_sim
0,546227,"[13, 12, 15, 5, 4]","[0.71266, 0.71266, 0.635631, 0.457258, 0.457258]"
1,546408,"[470, 45, 434, 28, 29]","[0.760274, 0.758144, 0.675339, 0.617515, 0.617515]"
2,546234,"[18, 478, 477, 281, 282]","[0.928725, 0.650948, 0.650948, 0.180814, 0.159013]"
3,651258,"[404, 405, 403, 455, 211]","[0.700914, 0.700914, 0.66071, 0.188307, 0.165063]"
4,546355,"[39, 482, 321, 322, 286]","[0.910428, 0.608733, 0.466645, 0.466645, 0.397241]"


Сохраним его в файлы. `csv` для визуальной проверки сохраненных данных (в дальнейшем уберём), `json` для бэкенда.

In [30]:
df_result.to_csv(r'my_data.csv', index= False)

df_result.to_json(r'my_data', orient='index')

Данный файл будет передаваться в бэкенд один раз при загрузке программы.<br>
Проверим точность работы функции `predict`, использовав полученные данные.

## Валидация решения

Для проверки точности нашего решения создадим функцию `metric`, которая будет учитывать только попадание в топ-5 правильного названия заказчика, пока без учёта позиции. В дальнейшем будет введена метрика `mean_reciprocal_rank`.

In [31]:
def metric(actual, pred):
    count = 0
    for i in range(len(actual)):
        if actual[i] in pred[i]:
            count += 1
    
    return round(count /len(actual), 4)

Для проверки метрики воспользуемся датасетом `df_productdealerkey`.

In [32]:
df_productdealerkey[['key', 'product_id']].head()

,key,product_id
0,546227,12
1,651265,106
2,546257,200
3,546408,38
4,651258,403


In [33]:
df = df_result.merge(df_productdealerkey[['key', 'product_id']], left_on='product_key', right_on='key')

In [34]:
df.head()

,product_key,product_id_x,pred_sim,key,product_id_y
0,546227,"[13, 12, 15, 5, 4]","[0.71266, 0.71266, 0.635631, 0.457258, 0.457258]",546227,12
1,546408,"[470, 45, 434, 28, 29]","[0.760274, 0.758144, 0.675339, 0.617515, 0.617515]",546408,38
2,546234,"[18, 478, 477, 281, 282]","[0.928725, 0.650948, 0.650948, 0.180814, 0.159013]",546234,18
3,651258,"[404, 405, 403, 455, 211]","[0.700914, 0.700914, 0.66071, 0.188307, 0.165063]",651258,403
4,546355,"[39, 482, 321, 322, 286]","[0.910428, 0.608733, 0.466645, 0.466645, 0.397241]",546355,39


In [35]:
metric(df.product_id_y, df.product_id_x)

0.8687

**0.8687** - 87% правильных названий заказчиков попали в топ-5 (пока без учёта позиции!). Без блока с предобработкой текста данный параметр ровнялся 76%. Но пока всё равно не 100. Проверим позиции, по которым наша функция `predict` выдала ошибочные предсказания.

## Анализ ошибочных предсказаний

Создадим и запустим функцию `first_n_no_match` для визуализации `n` первых ошибочных предсказаний.

In [36]:
def first_n_no_match(actual, pred, pred_sim, n):
    count = 0
    for i in range(n):
        if actual[i] not in pred[i]:
            print('i', i ,'actual' , actual[i], 'pred', pred[i], 'pred_sim', pred_sim[i])
            count += 1
    return count
    
print(first_n_no_match(df.product_id_y, df.product_id_x, df.pred_sim,  600))

i 1 actual 38 pred [470, 45, 434, 28, 29] pred_sim [0.760274, 0.758144, 0.675339, 0.617515, 0.617515]
i 17 actual 307 pred [259, 260, 261, 262, 306] pred_sim [0.484167, 0.484167, 0.44935, 0.432372, 0.397735]
i 19 actual 234 pred [496, 229, 228, 485, 232] pred_sim [0.385151, 0.261053, 0.261053, 0.260991, 0.244707]
i 25 actual 276 pred [271, 272, 273, 275, 274] pred_sim [0.789599, 0.740264, 0.711377, 0.700744, 0.672044]
i 29 actual 235 pred [496, 229, 228, 485, 232] pred_sim [0.385151, 0.261053, 0.261053, 0.260991, 0.244707]
i 42 actual 55 pred [53, 54, 50, 51, 52] pred_sim [0.438065, 0.438065, 0.423054, 0.423054, 0.420493]
i 43 actual 52 pred [54, 53, 50, 51, 57] pred_sim [0.37315, 0.37315, 0.360363, 0.360363, 0.358441]
i 63 actual 332 pred [336, 335, 334, 350, 349] pred_sim [0.593076, 0.593076, 0.593076, 0.589857, 0.589857]
i 64 actual 356 pred [336, 335, 334, 350, 349] pred_sim [0.593076, 0.593076, 0.593076, 0.589857, 0.589857]
i 65 actual 341 pred [336, 335, 334, 350, 349] pred_sim [

Посмотрим на позицию `i 42 actual 55 pred [53, 54, 50, 51, 52] pred_sim [0.438065, 0.438065, 0.423054, 0.423054, 0.420493]`. Много предсказаний являются соседями. Так же смущают одинаковое сходство для разных позиций.

In [37]:
df.iloc[42]

product_key                                                 716500
product_id_x                                  [53, 54, 50, 51, 52]
pred_sim        [0.438065, 0.438065, 0.423054, 0.423054, 0.420493]
key                                                         716500
product_id_y                                                    55
Name: 42, dtype: object

In [38]:
df_product[df_product.id.isin(df.iloc[42].product_id_x)]

,id,name
id_unique,,
313,50,средство удаления ржавчины минеральных отложений щадящего действия bath acid концентрат 1 200 1 500 1 л
314,53,средство усиленного действия удаления ржавчины минеральных отложений bath acid концентрат 1 200 1 500 1 л
428,51,средство удаления ржавчины минеральных отложений щадящего действия bath acid концентрат 1 200 1 500 5 л
435,54,средство усиленного действия удаления ржавчины минеральных отложений bath acid концентрат 1 200 1 500 5 л
486,52,средство усиленного действия удаления ржавчины минеральных отложений bath acid концентрат 1 200 1 500 0 75 л


In [39]:
df_dealerprice[df_dealerprice.product_key == df.iloc[42].product_key]

,product_key,product_name
42,716500,средство удаления ржавчины минеральных отложений 0 75л bath acid prosept цитрус 294 075


Это позиции заказчика для похожих товаров, которые отличаются объёмом и парой слов. А сходство идентично в тез позициях, где разница лишь в цифре объёма. Видимо наш метод не очень хорошо отличает числа.

Теперь посмотрим на позицию `i 503 actual 406 pred [52, 126, 176, 180, 175] pred_sim [0.0, 0.0, 0.0, 0.0, 0.0]`

In [40]:
df.iloc[503]

product_key                    1001472194
product_id_x     [52, 126, 176, 180, 175]
pred_sim        [0.0, 0.0, 0.0, 0.0, 0.0]
key                            1001472194
product_id_y                          406
Name: 503, dtype: object

In [41]:
df_product[df_product.id.isin(df.iloc[503].product_id_x)]

,id,name
id_unique,,
153,176,жидкое гель мыло эконом класса без красителей ароматизаторов diona e готовый состав 5 л
154,180,пенное мыло дозаторов цветочным ароматом diona aroma концентрат 5 л
155,175,жидкое гель мыло эконом класса без красителей ароматизаторов diona e готовый состав 5 л пэт
181,126,средство комплексного мытья отбеливания поверхностей дезинфицирующим эффектом duty belizna концентрат 1 200 1 3000 5 л
486,52,средство усиленного действия удаления ржавчины минеральных отложений bath acid концентрат 1 200 1 500 0 75 л


In [42]:
df_dealerprice[df_dealerprice.product_key == df.iloc[503].product_key]

,product_key,product_name
456,1001472194,добавкапротивплесени prosept041 025 0 25л
4131,1001472194,добавка против плесени prosept 041 025 0 25 л


Данная позиция имеет дубликат по `product_key`, и является двумя видами написания одного товара. При этом сходство данного товара равно нулю к любой позиции заказчика. Посмотрим, есть ли соответствие названию заказчика через таблицу с результатами ручного парсинга.

In [43]:
df_productdealerkey[df_productdealerkey.key == df.iloc[503].product_key].product_id

1444    406
Name: product_id, dtype: int64

In [44]:
df_product[df_product.id == 406]

,id,name
id_unique,,
478,406,добавка против появления плесени 0 25 л


Да, данная позиция от дилеров имеет чёткий мэтч. Внесём рассмотренные в этом разделе случаи в список к решению в дальнейшем.

## Выводы и планы

Релизована предобработка данных. Разработан метод сопоставления топ `n` названий заказчиков названиям дилеров. Получена отличная скорость выполнения кода и отличное значение метрики. В будущем необходимо выполнить:

1. Проработать вопрос об одинаковых названиях с разными `id`.
2. Оптимизировать блок кода с предобработкой текста.
3. Ввести метрику `mean_reciprocal_rank`.
4. Исследовать случаи, когда сходство = 0.
5. Добавить новые слова к функции добавления пробелов.
6. Применить `prediction` к эмбедингам от `tiny-BERT`.